In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
from itertools import product
from collections import defaultdict
from sqlalchemy import create_engine
from sklearn.feature_selection import RFE, chi2
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import sys
sys.path.append("../model")
from model import get_data, CancellationModel
from pipeline import Pipeline
import constants as C

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
engine = create_engine(C.ENGINE)
df = get_data(engine, booked=True)

In [4]:
with open('../model/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
df, X = model.pipeline.transform(df)

In [ ]:
df["insurance"] = df[["insurance_corporate", "insurance_personal", "insurance_silvercar"]].idxmax(axis=1)
    df["insurance"] = df["insurance"].map({"insurance_personal": "Personal", "insurance_corporate": "Corporate",
                                                   "insurance_silvercar": "Silvercar", None: "NA"})

In [15]:
df[["insurance_corporate", "insurance_personal", "insurance_silvercar"]].idxmax(axis=1)

0        insurance_corporate
1        insurance_corporate
2        insurance_corporate
3        insurance_corporate
4        insurance_corporate
5        insurance_corporate
6        insurance_corporate
7        insurance_corporate
8        insurance_corporate
9        insurance_corporate
10       insurance_corporate
11       insurance_corporate
12       insurance_corporate
13       insurance_corporate
14       insurance_corporate
15       insurance_corporate
16       insurance_corporate
17        insurance_personal
18       insurance_corporate
19       insurance_corporate
20        insurance_personal
21       insurance_corporate
22       insurance_corporate
23       insurance_corporate
24       insurance_corporate
25       insurance_corporate
26       insurance_corporate
27        insurance_personal
28       insurance_corporate
29        insurance_personal
                ...         
19687    insurance_corporate
19688     insurance_personal
19689    insurance_corporate
19690     insu

In [14]:
df[["insurance_corporate", "insurance_personal", "insurance_silvercar"]]

,insurance_corporate,insurance_personal,insurance_silvercar
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [13]:
df[["insurance_corporate", "insurance_personal", "insurance_silvercar"]].idxmax(axis=1)

0        insurance_corporate
1        insurance_corporate
2        insurance_corporate
3        insurance_corporate
4        insurance_corporate
5        insurance_corporate
6        insurance_corporate
7        insurance_corporate
8        insurance_corporate
9        insurance_corporate
10       insurance_corporate
11       insurance_corporate
12       insurance_corporate
13       insurance_corporate
14       insurance_corporate
15       insurance_corporate
16       insurance_corporate
17        insurance_personal
18       insurance_corporate
19       insurance_corporate
20        insurance_personal
21       insurance_corporate
22       insurance_corporate
23       insurance_corporate
24       insurance_corporate
25       insurance_corporate
26       insurance_corporate
27        insurance_personal
28       insurance_corporate
29        insurance_personal
                ...         
19687    insurance_corporate
19688     insurance_personal
19689    insurance_corporate
19690     insu